# jQuery

Let's first use Selenium to make a driver object

In [ ]:
from selenium import webdriver
# new driver (opens browser window)
driver = webdriver.Chrome(r'selenium_files/chromedriver.exe')

In [ ]:
# navigate to a page to work with
driver.get("https://www.startengine.com/explore")

## Download jQuery and save it to disk (as jquery.js)

Place it in the same folder as your notebook, or in a subfolder 'selenium_files'. (Or anywhere else, as long as you use the correct folder name when you read the file).

Url: https://code.jquery.com/jquery-3.6.4.js

## Read jquery from disk and 'inject' it (with execute_script)

Remember that execute_script is used to run Javascript in the browser.

In [ ]:
# this assumes subfolder 'selenium_files'
with open(r'selenium_files/jquery.js', 'r') as jquery_js:
    # read the file and then run it in the browser
    driver.execute_script( jquery_js.read() ) 

### Check it

At this point jQuery should be available in the browser. Open the debugger window (F12, or ctrl+C) and type '$.' (with the period, and without the quotes).

Typing the period will show autocomplete (the properties/functions that are available on that variable).

`$` is the function name of jQuery. Why `$` and not, say, 'kegerator'? Because `$` is a valid JavaScript character and less typing. 

If jQuery is correctly loaded, the first autocomplete option should be `_data` and if it is not loaded, the first option is `length`.

## Manual loading of jquery

If you don't want (need) to use Selenium to load jQuery, you can also add it manually to any web browser through the debugger.

Open a web page , and open the debug window (usually F12 or ctrl+shift+C).

Select the console, and copy/paste:

```javascript
var script = document.createElement("script")
script.type = "text/javascript";
script.src = "https://code.jquery.com/jquery-3.6.4.js";
document.getElementsByTagName("head")[0].appendChild(script);
```

The above code will load the jquery script. It's a bit convoluted, basically we are adding `<script type="text/javascript" src="https://code.jquery.com/jquery-3.6.4.js"></script>`,
and the browser will load and run the JavaScript at that url.

### now we can run jquery in the debugger 

#### Hyperlinks

> Note: it depends on the resolution of your screen. If it is wide, then make it small and refresh the page (F5).

Type the following jQuery in the console/debugger (it is not python and won't work from your notebook):
```
$("a")
```
There are 73 hyperlinks (click on the > symbol to expand). Click on '0' to see the properties (yes, a lot of properties).

The first one is 
```
$("a")[0]
```
Try hovering over the element in the console. It will 'light up' on the page. Yes, very helpful!

## How to get these elements to python?

Using `execute_script` we can actually get these 73 hyperlinks in python. Note that we have JavaScript that works (`$("a")`) so we are going to use that.

In [ ]:
# this is python code and you can run this in the notebook
# what is happening?
# Selenium runs JavaScript code that actually 'returns' something
# Since we are using `links =` whatever Selenium receives will be passed into that variable
links = driver.execute_script('return $("a")')

In [ ]:
# let's check the type of links and what it holds
print('type of links', type(links))
print('length of links', len(links))
print('type of links[0]', type(links[0]))

In [ ]:
# oh okay... yes, Selenium knows how to deal with WebElements! Great teamwork!!
print(links[0].get_attribute('href'))

### Enthusiastic instructor

Your enthusiastic instructor was bragging about the locator capabilities of jQuery. And now it is the time for him to deliver.

We don't want all the hyperlinks, we want the hyperlinks of the projects (when the page loads 12 projects are shown). So we want 12 project hyperlinks (and not all 73).

Inspect the HTML of a project's div (right-click and inspect).

Notice the container that holds all 12 projects: a div with a class MuiGrid-container. And that each of the projects is in a div with a class 'tombstone'.
    
In the console (not in the notebook), try this JavaScript:
    
```
$("div.tombstone a")
```

This first selects all divs with a class tombstone, and then gets all hyperlinks (element a) within that div. Note the period indicates a class:

Some subtle things:
```
div.tombstone: a div with class tombstone
        
div .tomstone: a div with a child (grand child, etc) with class tombstone
```

Much better; we went down from 73 hyperlinks to having 2 x 12 hyperlinks. We can either fix that in python (list(set(list))), or fix it in jQuery.

To fix this with jQuery we need to inspect the HTML to find something 'unique' or 'different' that we can leverage. There is no way to know unless we inspect the HTML.

Inspect and notice each tombstone has 4 divs. Our problem is that the first two divs each have the link. But, the first div has a class that the second one does not have ('activeEvent').

So, we found something different, so let's leverage that! The following selector will require class 'activeEvent', and that makes the match unique.

```
$("div.tombstone .activeEvent a")
```

Meaning: find divs with a class tombstone, within that div an element with a class activeEvent, and within that element the hyperlink (a)

In [ ]:
# let's bring these to pyton
links = driver.execute_script('return $("div.tombstone .activeEvent a")')

In [ ]:
# let's check the type of links and what it holds
print('type of links', type(links))
print('length of links', len(links))
print('type of links[0]', type(links[0]))

In [ ]:
for link in links[0:3]:
    print(link.get_attribute('href'))

### What if we want to pass the hrefs (instead of the whole element)

This gives a list of a elements (what we had earlier)
```
$("div.tombstone .activeEvent a")
```

This puts the list of elements into a JavaScript array (list)

```
els = $("div.tombstone .activeEvent a")
```
You can pass in any of these elements to jQuery to use jQuery functions; in this case `attr` (gives access to attributes) (this is the equivalent of get_attribute in Selenium)
```
$(els[0]).attr("href")
```

Take each of the elements, and pass them into `each`. The `each` function will take one element (a) at the time and pass it into the anonymous function that is passed to each.
The anonymous function (`function() { links.push( $(this).attr('href')) }`) will append (push in JavaScript) the href onto the array (list).

`this` refers to the current element (one element at a time), so `$(this)` allows us to use jQuery on each element.

```
links = []
$("div.tombstone .activeEvent a").each( function() { links.push( $(this).attr('href')) }   )
```

Can we get a one-liner? Yes, with the map function we can control what is being returned:

```
$("div.tombstone .activeEvent a").map( function() { return( $(this).attr("href")) } )
```

In [ ]:
# python
links = driver.execute_script('return $("div.tombstone .activeEvent a").map( function() { return( $(this).attr("href")) } )')

In [ ]:
links[0:3]

### `map` vs `each`

The `each` method is meant to be an immutable iterator, where as the `map` method can be used as an iterator, but is really meant to manipulate the supplied array and return a new array.

Another important thing to note is that the each function returns the original array while the map function returns a new array. 

See: https://stackoverflow.com/questions/749084/jquery-map-vs-each
        

## Token, price, 24hr change from coinmarketcap.com

In [ ]:
# use a function to prevent copy-paste the same code
# url: string of url
def loadUrl( url ):
    # navigate to a page to work with
    driver.get( url )
    # this assumes subfolder 'selenium_files'
    with open(r'selenium_files/jquery.js', 'r') as jquery_js:
        # read the file and then run it in the browser
        driver.execute_script( jquery_js.read() )

In [ ]:
# use our function to go to coinmarketcap.com and inject jQuery
loadUrl("https://www.coinmarketcap.com")

#### Inspect the HTML, see the table with class 'cmc-table'

```
els = $("table.cmc-table tr")
```

The first row is the header, https://stackoverflow.com/questions/2259393/jquery-select-all-except-first
```
els = $("table.cmc-table tr:not(:first)")
```

## jQuery `find`

In Selenium we could use the locator functions on the driver (whole document), or on elements (to just look 'inside' the element).

Likewise, jQuery has a `find` function, that you can use on an element (so it will look inside that element only).

For example: `$(els[1]).find("td")` -> `$(els[1])` takes an element and will allow us to use jQuery functions (in this case `find`). 
    
We are going to find all `td` elements (wich are table cells). In other words, we feed it a row and will get a list of cells. Why? Because there are certains cells we are interested in (we are going to get the token name, the price and the 24 hour % change). Since a table is predictable/structured, for example, the project name is always in the third cell, this will help us get what we need.


#### Each tr (table row) has td elements (cells)

```
cells = $(els[1]).find("td") 

```
cells[2]: token name, cells[3]: price, cells[4]: 24h %

inspect cells[2], notice p with color="text" holding name, and p color="text3" holding abbreviation

```
$(cells[2]).find('p[color="text"]').html()
$(cells[2]).find('p[color="text3"]').html()
```

### Let's get the details with a function
```
function getDetails( tr ) {
    cells = $(tr).find("td") 
    return {
        "name" : $(cells[2]).find('p[color="text"]').html() ,
        "slug" : $(cells[2]).find('p[color="text3"]').html() ,
        "price" : $(cells[3]).text(),
        "24h" : $(cells[4]).text()
    }
};
console.log( getDetails (els[1]) );
```

#### Go through all elements (rows) and retrieve the details
```
details = []
els.each( function(i, el) { details.push(getDetails(el))} )
```

or

```
details = $("table.cmc-table tr:not(:first)").map( function() { return(getDetails(this))} );
```

## Bringing JavaScript variables 'home' (to Selenium)

In [ ]:
# this is python code; getDetails is a python string, holding JavaScript
# in the next cell we are going to 'feed' this into the browser
getDetails = '''
/*
This function gets a table row (tr) element
It will pass it into jQuery to be able to 'look inside'
It will return an object (dictionary) with the name, slug (short name), price and 24h price change
*/
function getDetails( tr ) {
    cells = $(tr).find("td") 
    return {
        "name" : $(cells[2]).find('p[color="text"]').html() ,
        "slug" : $(cells[2]).find('p[color="text3"]').html() ,
        "price" : $(cells[3]).text(),
        "24h" : $(cells[4]).text()
    };
};

/*
This code gets all table rows (except the first) and uses the map functionality to make a list of dictionaries
(each dictionary is made by function getDetails)
*/
return $("table.cmc-table tr:not(:first)").map( function() { return(getDetails(this))} );
'''

In [ ]:
# run the JavaScript and set the returned value into 'det' 
det = driver.execute_script(getDetails)

In [ ]:
det

In [ ]:
print('type of det:', type (det))
print('elements are type: ', type(det[0]))

### What's up with the empty dictionaries? 

Lazy loading!! The table only gets populated when the user scrolls down

> How can we trigger scrolling down? 